# Homography

This notebook explains

1. how to estimate the homography matrix from 2D-2D point correspondences, and
2. how to calculate the homography matrix given the projection matrices and the plane parameters (i.e., the normal and the distance).


## Libraries

In [1]:
import sys, os, cv2
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.insert(0,module_path)

from pycalib.calib import lookat
from pycalib.homography import calc_homography_wcs

## Synthetic data

This cell defines a set of calibration parameters that satisfy

- world-to-camera: $c_i = R_i X + t_i$, and
- camera-to-image: $\lambda \hat{x}_i = K_i c_i$,

where $X$ is a 3D point in the world coodinate system, and $i = 1,2$.

In [2]:
# 3D points on a plane

# a random point on the plane
X0 = (np.random.rand(1, 3) - 0.5)*5 + [0, 0, 10]
X0 = X0.reshape((3,1))

# normal vector
n_gt = np.random.uniform(-1, 1, size=3)
n_gt = n_gt / np.linalg.norm(n_gt)
n_gt = n_gt.reshape((3,1))
d_gt = - n_gt.T @ X0

# an axis not parallel to n_gt
a = np.array([1.0, 0.0, 0.0])
if np.allclose(np.cross(a, n_gt.flatten()), 0):
    a = np.array([0.0, 1.0, 0.0])
# orthonormal axes on the plane
u_gt = np.cross(n_gt.flatten(), a)
u_gt = u_gt / np.linalg.norm(u_gt)
v_gt = np.cross(n_gt.flatten(), u_gt)

# Random points on the plane
N = 100
alphas = np.random.uniform(-5, 5, size=N)
betas  = np.random.uniform(-5, 5, size=N)
X_gt = X0.flatten() + np.outer(alphas, u_gt) + np.outer(betas, v_gt)


# Camera poses
R1_gt = np.eye(3)
t1_gt = np.zeros(3).reshape((3,1))
R2_gt, t2_gt = lookat(np.array([1,0,0]), np.array([0,0,10]), np.array([0,1,0]))
rvec1_gt = cv2.Rodrigues(R1_gt)[0]
rvec2_gt = cv2.Rodrigues(R2_gt)[0]

# Camera intrinsics
K1 = np.array([[600, 0, 320], [0, 600, 240], [0, 0, 1]]).astype(np.float64)  # VGA camera
K2 = np.array([[800, 0, 640], [0, 800, 360], [0, 0, 1]]).astype(np.float64)  # 720p camera

# 2D corresponding points
x1 = cv2.projectPoints(X_gt.reshape((-1, 1, 3)), rvec1_gt, t1_gt, K1, None)[0].reshape((-1, 2))
x2 = cv2.projectPoints(X_gt.reshape((-1, 1, 3)), rvec2_gt, t2_gt, K2, None)[0].reshape((-1, 2))

# Verify triangulation
Y = cv2.triangulatePoints(K1 @ np.hstack((R1_gt, t1_gt)), K2 @ np.hstack((R2_gt, t2_gt)), x1.T, x2.T)
Y = Y[:3] / Y[3,:]
assert np.allclose(0, X_gt - Y.T)

# Verify z > 0 at each camera
assert np.all(X_gt[:, 2] > 0)
assert np.all((R2_gt @ X_gt.T + t2_gt)[2, :] > 0)

## Homography from corresponding points

In [3]:
# estimate H from n point correspondences
n = 4 # 4 or more
H, mask = cv2.findHomography(x1[:n], x2[:n])
H = H/H[2,2]
print(H)

# verify if H can map 2D points from 1 to 2
x1h = cv2.convertPointsToHomogeneous(x1).reshape((-1,3)).T
y2 = H @ x1h
y2 = y2 / y2[2,:]
y2 = (y2.T)[:,:2]

assert np.allclose(x2, y2, rtol=1e-3), f'{np.linalg.norm(x2-y2, axis=1)}'


[[ 1.00626516e+00  1.46083114e-02  2.66781901e+02]
 [-4.92195419e-02  1.26410724e+00  5.64427504e+01]
 [-1.36721128e-04 -1.98481103e-06  1.00000000e+00]]


## Compute $H$ from projection matrices and the plane parameters

Suppose we know the plane parameter $\boldsymbol{n}$ and $d$ satisfying
\begin{equation}
\boldsymbol{n}^\top \boldsymbol{X} + d = 0 \,,
\end{equation}
and know the projection matrices of the two cameras as
\begin{equation}
\begin{split}
\lambda_1 \boldsymbol{x}_1 & = K_1 ( R_1 \boldsymbol{X} + \boldsymbol{t}_1 ) \,, \\
\lambda_2 \boldsymbol{x}_2 & = K_2 ( R_2 \boldsymbol{X} + \boldsymbol{t}_2 ) \,,
\end{split}
\end{equation}
where $\lambda_1$ and $\lambda_2$ are the depths from the camera centers.

By using the 1st camera as the world coordinate system, the above can be rewritten as
\begin{equation}
\begin{split}
\lambda_1 \boldsymbol{x}_1 & = K_1 \boldsymbol{X}_1 \,, \\
\lambda_2 \boldsymbol{x}_2 & = K_2 ( R_{12} \boldsymbol{X}_1 + \boldsymbol{t}_{12} ) \,, \\
\boldsymbol{n}_1^\top \boldsymbol{X}_1 + d_1 & = 0 \,,
\end{split}
\end{equation}
where $R_{12} = R_2 R_1^\top$ and $\boldsymbol{t}_{12} = \boldsymbol{t}_2 - R_2 R_1^\top \boldsymbol{t}_1$, $\boldsymbol{n}_1 = \boldsymbol{n} R_1^\top$, and $d_1 = d - \boldsymbol{n}^\top R_1^\top \boldsymbol{t}$.

The first equation gives us
\begin{equation}
\boldsymbol{X}_1 = \lambda_1 K_1^{-1} \boldsymbol{x}_1 \,,
\end{equation}
and if this point is on the plane, it satisfies
\begin{equation}
\begin{split}
& \boldsymbol{n}_1^\top \boldsymbol{X}_1 + d_1 = 0 \,, \\
\Leftrightarrow \quad &
\boldsymbol{n}_1^\top \left(\lambda_1 K_1^{-1} \boldsymbol{x}_1\right) + d_1 = 0 \,, \\
\Leftrightarrow \quad &
\lambda_1 = -\frac{d_1}{\boldsymbol{n}_1^\top K_1^{-1} \boldsymbol{x}_1} \,.
\end{split}
\end{equation}

By plugging this $\lambda_1$ and $\boldsymbol{X}_1 = \lambda_1 K_1^{-1} \boldsymbol{x}_1$ to the projection equation, we obtain
\begin{equation}
\begin{split}
\boldsymbol{x}_2 & \sim K_2 \left( R_{12} \boldsymbol{X}_1 + \boldsymbol{t}_{12} \right) \,, \\
\boldsymbol{x}_2 & \sim K_2 \left( \lambda_1 R_{12} K_1^{-1} \boldsymbol{x}_1 + \boldsymbol{t}_{12} \right) \,, \\
\boldsymbol{x}_2 & \sim K_2 \left( -\frac{d_1}{\boldsymbol{n}_1^\top K_1^{-1} \boldsymbol{x}_1} R_{12} K_1^{-1} \boldsymbol{x}_1 + \boldsymbol{t}_{12} \right) \,, \\
\boldsymbol{x}_2 & \sim - K_2 \frac{d_1 R_{12} K_1^{-1} \boldsymbol{x}_1 - \boldsymbol{t}_{12} \boldsymbol{n}_1^\top K_1^{-1} \boldsymbol{x}_1}{\boldsymbol{n}_1^\top K_1^{-1} \boldsymbol{x}_1} \,, \\
\boldsymbol{x}_2 & \sim - K_2 \frac{d_1 R_{12} - \boldsymbol{t}_{12} \boldsymbol{n}_1^\top}{\boldsymbol{n}_1^\top K_1^{-1} \boldsymbol{x}_1} K_1^{-1} \boldsymbol{x}_1 \,, \\
\boldsymbol{x}_2 & \sim - \frac{d_1}{\boldsymbol{n}_1^\top K_1^{-1} \boldsymbol{x}_1} K_2 (R_{12} - \frac{\boldsymbol{t}_{12} \boldsymbol{n}_1^\top}{d_1}) K_1^{-1} \boldsymbol{x}_1 \,, \\
\boldsymbol{x}_2 & \sim K_2 \left(R_{12} - \frac{\boldsymbol{t}_{12} \boldsymbol{n}_1^\top}{d_1} \right) K_1^{-1} \boldsymbol{x}_1 \,.
\end{split}
\end{equation}
Notice that $- \frac{d_1}{\boldsymbol{n}_1^\top K_1^{-1} \boldsymbol{x}_1}$ is a scalar value and can be omitted since $\sim$ denotes equality up to an arbitrary non-zero scale factor.

That is, the homography matrix satisfing $\boldsymbol{x}_2 \sim H \boldsymbol{x}_1$ is given by
\begin{equation}
H = K_2 \left(R_{12} - \frac{\boldsymbol{t}_{12} \boldsymbol{n}_1^\top}{d_1} \right) K_1^{-1} \,.
\end{equation}

In plactice, $H$ can be normalized by its Frobenius norm or by `H[2,2]` as done in `cv2.findHomography()`.


In [4]:
# calc R12, t12
R12 = R2_gt @ R1_gt.T
t12 = t2_gt - R12 @ t1_gt
n1 = (n_gt.T @ R1_gt.T).T
d1 = d_gt - n_gt.T @ R1_gt.T @ t1_gt

# calc H
H = K2 @ (R12 - (t12 @ n1.T / d1)) @ np.linalg.inv(K1)
H = H / H[2,2]
print(H)

# verify if H can map 2D points from 1 to 2
x1h = cv2.convertPointsToHomogeneous(x1).reshape((-1,3)).T
y2 = H @ x1h
y2 = y2 / y2[2,:]
y2 = (y2.T)[:,:2]

assert np.allclose(x2, y2, rtol=1e-3), f'{np.linalg.norm(x2-y2, axis=1)}'


[[ 1.00626548e+00  1.46082767e-02  2.66781874e+02]
 [-4.92195072e-02  1.26410740e+00  5.64427358e+01]
 [-1.36720853e-04 -1.98482021e-06  1.00000000e+00]]


In [5]:
# the same
H = calc_homography_wcs(K1, R1_gt, t1_gt, K2, R2_gt, t2_gt, n_gt, d_gt)
print(H)

# verify if H can map 2D points from 1 to 2
x1h = cv2.convertPointsToHomogeneous(x1).reshape((-1,3)).T
y2 = H @ x1h
y2 = y2 / y2[2,:]
y2 = (y2.T)[:,:2]

assert np.allclose(x2, y2, rtol=1e-3), f'{np.linalg.norm(x2-y2, axis=1)}'


[[ 3.69009162e-03  5.35702364e-05  9.78319913e-01]
 [-1.80493612e-04  4.63562769e-03  2.06982024e-01]
 [-5.01371145e-07 -7.27856472e-09  3.66711538e-03]]
